# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import csv
import billboard
import json
import re
import lyricsgenius
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials

### Genius and Spotify APIs' settings

In [2]:
# Genius token
genius_token = os.environ.get('genius_token')

# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_client_secret)
sp = sp.Spotify(client_credentials_manager=client_credentials_manager)

### CSV data

In [3]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
wd = os.getcwd()
data_path = os.chdir(wd + '\data')
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0, sep = ';')
    year = os.path.basename(filename)[:-4]
    songs['Year'] = year
    songs_50_15.append(songs)

### billboard library data

In [4]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [5]:
# Preprocessing data from years 2016-2020
song_titles, artists = [[] for i in range(2)]

for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(' by ')
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

### Concatenating CSV and billboard library data

In [6]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))

# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dictionary = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dictionary)
        songs.append(as_pandas_df)
    return songs

# Concatenate all songs from years 1950 - 2020
songs_16_20 = dicts_of_dates(dates)
all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020
6676,97,Luke Combs,Lovin' On You,2020
6677,98,Moneybagg Yo,Said Sum,2020
6678,99,H.E.R. Featuring YG,Slide,2020


In [7]:
# all_time_billboard_wrap_up dataframe nans checking
nans_table = [all_time_billboard_wrap_up['Position'].isna().any(), all_time_billboard_wrap_up['Artist'].isna().any(), all_time_billboard_wrap_up['Song Title'].isna().any(), all_time_billboard_wrap_up['Year'].isna().any()]
nans_proj = pd.DataFrame({'Column': ['Position', 'Artist', 'Song Title', 'Year'], 'Is nan?': nans_table}).set_index('Column')
display(nans_proj)

,Is nan?
Column,
Position,False
Artist,False
Song Title,False
Year,False


In [8]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Spliting authors by 'and' and 'Featuring' to make api searching easier and more efficient
artists = list(all_time_billboard_wrap_up.loc[:,'Artist'])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|Featuring|feat.|&', str(artist))):
        result = re.split(r'and|Featuring|feat.|&', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up['Split Names'] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,Split Names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,Gary
...,...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020,Morgan Wallen
6676,97,Luke Combs,Lovin' On You,2020,Luke Combs
6677,98,Moneybagg Yo,Said Sum,2020,Moneybagg Yo
6678,99,H.E.R. Featuring YG,Slide,2020,H.E.R.


### Genius data into json files

In [9]:
# lyricsgenius settings
genius = lyricsgenius.Genius(genius_token, timeout = 30, retries = 3)
songs = list(all_time_billboard_wrap_up.loc[:, 'Song Title'])
artists = list(all_time_billboard_wrap_up.loc[:,'Split Names'])
jsons_path = wd + '\\jsons\\'

# Saving lyrics to jsons function
def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# No lyrics function (only instrumental songs)
def no_lyrics(json_name):
    with open(jsons_path + json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': ""}, f, ensure_ascii = False, indent = 4)  

In [10]:
# Applying return_lyrics_json and no_lyrics functions on lyrics:
def json_data():
    error_list = []
    for i in range(len(songs)):
        try:
            return_lyrics_json(songs[i], artists[i], str(i)+'.json')
        except AttributeError:
            no_lyrics(str(i)+'.json')
            error_list.append(i)
# json_data()

In [11]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# err based on error_list
# Fixed 6140 Beyonce 7/11 (date format in csv)
err = [12, 23, 25, 53, 60, 74, 84, 100, 101, 103, 111, 116, 141, 160, 173, 190, 196, 210, 233, 239, 266, 277, 334, 365, 380, 385, 386, 401, 428, 439, 440, 448, 501, 515, 519, 537, 554, 565, 604, 616, 619, 633, 690, 694, 696, 708, 723, 771, 774, 784, 800, 816, 832, 838, 853, 862, 866, 871, 906, 920, 937, 969, 991, 1039, 1111, 1126, 1150, 1180, 1211, 1245, 1277, 1381, 1387, 1397, 1400, 1422, 1454, 1537, 1542, 1569, 1598, 1613, 1615, 1668, 1801, 1807, 1822, 1850, 1873, 1895, 1940, 1965, 1982, 2027, 2095, 2144, 2167, 2177, 2189, 2208, 2269, 2350, 2370, 2400, 2528, 2559, 2603, 2633, 2690, 2738, 2791, 2835, 2843, 3140, 3334, 3460, 3742, 3908, 3952, 4121, 4169, 4171, 4180, 4208, 4244, 4245, 4280, 4351, 4361, 4387, 4518, 4579, 4760, 4777, 4823, 5134, 5235]

In [12]:
# Deleting empty json files
# Adding string '.json' to err
json_delete = [str(j) + '.json' for j in err]
# Deteling json files that are in json_delete list and bigger than 5KB
all_json_files = os.listdir(jsons_path)
for x in all_json_files:
    filepath = jsons_path + str(x)
    json_size = os.path.getsize(filepath)
    if (json_size > 5 * 1024) or (x in err): #error_list
        os.remove(filepath)
# Selecting all_billboard_wrap_up data corresponding with actual list of jsons
all_time_billboard_wrap_up_cleaned = all_time_billboard_wrap_up.iloc[[int(x[:-5]) for x in os.listdir(jsons_path)]]

# need correct order

In [13]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Function that enables to find Spotify ID
def find_spotify_id(artist, track):
    try:
        track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        return track_id['tracks']['items'][0]['id']
    except IndexError:
        return ''    
all_time_billboard_wrap_up_cleaned['Spotify ID'] = [find_spotify_id(all_time_billboard_wrap_up_cleaned['Split Names'].iloc[i], all_time_billboard_wrap_up_cleaned['Song Title'].iloc[i]) for i in range(len(all_time_billboard_wrap_up_cleaned))]
display(all_time_billboard_wrap_up_cleaned)

<ipython-input-13-64f91ea8b3f3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_time_billboard_wrap_up_cleaned['Spotify ID'] = [find_spotify_id(all_time_billboard_wrap_up_cleaned['Split Names'].iloc[i], all_time_billboard_wrap_up_cleaned['Song Title'].iloc[i]) for i in range(len(all_time_billboard_wrap_up_cleaned))]


,Position,Artist,Song Title,Year,Split Names,Spotify ID
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins,1fhLgOJgIIZEsWWffk8ljs
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole,5dae01pKNjRQtgOeAkFzPY
10,11,Eileen Barton,If I Knew You Were Coming I'd've Baked A Cake,1950,Eileen Barton,
100,11,Pee Wee Hunt,Oh,1953,Pee Wee Hunt,28OFTSxTmCNj8O4hgyifjn
1000,21,Dixie Cups,Chapel Of Love,1964,Dixie Cups,3FYKl1WT0MW31lrwYgiMWB
...,...,...,...,...,...,...
995,16,Beatles,Please Please Me,1964,Beatles,15tcqNCx9skvXmmlscpAjF
996,17,Martha and The Vandellas,Dancing In The Street,1964,Martha,6rLqjzGV5VMLDWEnuUqi8q
997,18,Billy J. Kramer and The Dakotas,Little Children,1964,Billy J. Kramer,1i9KZctiFjJwuD2LFTThqN
998,19,Ray Charles Singers,Love Me With All Your Heart,1964,Ray Charles Singers,7F9PBz5Dqqe9gmM2n2g8yv


In [14]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
sum(all_time_billboard_wrap_up_cleaned['Spotify ID'] == '')

556

In [15]:
# Check artist and song name for n Spotify IDs
spotify_ids = all_time_billboard_wrap_up_cleaned['Spotify ID'].tolist()
spotify_artists = []
spotify_songs = []
for i in range(0, len(spotify_ids)):
    if spotify_ids[i] != '':
        track = sp.track(spotify_ids[i])
        song_name = track['name']
        spotify_songs.append(song_name)
        artist = track['album']['artists'][0].get('name')
        spotify_artists.append(artist)
    else:
        song_name, artist = '', ''
        spotify_songs.append(song_name)
        spotify_artists.append(artist)

In [16]:
all_time_billboard_wrap_up_cleaned['Spotify Artist'] = spotify_artists
all_time_billboard_wrap_up_cleaned['Spotify Song'] = spotify_songs
display(all_time_billboard_wrap_up_cleaned)

<ipython-input-16-391bca80b381>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_time_billboard_wrap_up_cleaned['Spotify Artist'] = spotify_artists
<ipython-input-16-391bca80b381>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_time_billboard_wrap_up_cleaned['Spotify Song'] = spotify_songs


,Position,Artist,Song Title,Year,Split Names,Spotify ID,Spotify Artist,Spotify Song
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins,1fhLgOJgIIZEsWWffk8ljs,The Weavers,Goodnight Irene
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa
10,11,Eileen Barton,If I Knew You Were Coming I'd've Baked A Cake,1950,Eileen Barton,,,
100,11,Pee Wee Hunt,Oh,1953,Pee Wee Hunt,28OFTSxTmCNj8O4hgyifjn,Pee Wee Hunt,Oh!
1000,21,Dixie Cups,Chapel Of Love,1964,Dixie Cups,3FYKl1WT0MW31lrwYgiMWB,The Dixie Cups,Chapel Of Love
...,...,...,...,...,...,...,...,...
995,16,Beatles,Please Please Me,1964,Beatles,15tcqNCx9skvXmmlscpAjF,The Beatles,Let It Be / Please Please Me / Let It Be - Tak...
996,17,Martha and The Vandellas,Dancing In The Street,1964,Martha,6rLqjzGV5VMLDWEnuUqi8q,Martha Reeves & The Vandellas,Dancing In The Street - Stereo
997,18,Billy J. Kramer and The Dakotas,Little Children,1964,Billy J. Kramer,1i9KZctiFjJwuD2LFTThqN,Billy J. Kramer,Little Children
998,19,Ray Charles Singers,Love Me With All Your Heart,1964,Ray Charles Singers,7F9PBz5Dqqe9gmM2n2g8yv,Ray Charles Singers,Love Me With All Your Heart (Cuando Caliente E...


In [17]:
all_time_billboard_wrap_up_cleaned[all_time_billboard_wrap_up_cleaned['Spotify Artist'].isin(all_time_billboard_wrap_up_cleaned['Artist'])]

,Position,Artist,Song Title,Year,Split Names,Spotify ID,Spotify Artist,Spotify Song
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole,5dae01pKNjRQtgOeAkFzPY,Nat King Cole,Mona Lisa
100,11,Pee Wee Hunt,Oh,1953,Pee Wee Hunt,28OFTSxTmCNj8O4hgyifjn,Pee Wee Hunt,Oh!
1001,22,Terry Stafford,Suspicion,1964,Terry Stafford,17YtDVU9bUluyihdOKWfxu,Terry Stafford,Suspicion
1006,27,Gene Pitney,It Hurts To Be In Love,1964,Gene Pitney,0Sr9pmp8QMts6l3kAunRrP,Gene Pitney,It Hurts To Be In Love
1013,34,Betty Everett and Jerry Butler,Let It Be Me,1964,Betty Everett,4rmB4N9wNLiq18OT40mZvd,Various Artists,Let It Be Me - Live in Pittsburg
...,...,...,...,...,...,...,...,...
986,7,Mary Wells,My Guy,1964,Mary Wells,4591VqUIXysNlmI5NcAIUd,Mary Wells,My Guy
99,10,Frankie Laine,I Believe,1953,Frankie Laine,3FzkbFgVGdg4zqyCMzUPM7,Frankie Laine,I Believe
991,12,Al Hirt,Java,1964,Al Hirt,5MdHvAvlmQJ3JHIvIK9uXz,Al Hirt,Java
994,15,Manfred Mann,Do Wah Diddy Diddy,1964,Manfred Mann,69sEXPrKQ5PtCxel5csSXs,Manfred Mann,Do Wah Diddy Diddy - 2007 Remaster
